In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import random

In [251]:
block_size = 32
batch_size = 16
dropout = 0.0

n_embd = 64
n_head = 2
n_layer = 4

max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
eval_iters = 200

In [205]:
with open('../dataset/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [252]:
class Head(nn.Module):
    """ one head of self attention"""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)

        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2, -1) * C**-0.5 # (B,T,C) @ (B, C, T) = (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))
        wei = F.softmax(wei, dim=1)
        wei = self.dropout(wei)

        #perform the weighted aggregation of the values
        v = self.value(x) # B,T,C
        out = wei @ v # (B,T,T) @ (B,T,C) => (B,T,C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self attention in parallel"""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([ Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity"""

    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd, n_embd),
            nn.Dropout(dropout),
        )
    
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


In [253]:
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
    

    def forward(self, idx, targets=None):
        B, T = idx.shape # B=16, T=3

        tok_embd = self.token_embedding_table(idx)
        pos_embd = self.position_embedding_table(torch.arange(T))
        x = tok_embd + pos_embd
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


In [254]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y


In [255]:
train_data.shape, val_data.shape

(torch.Size([1003854]), torch.Size([111540]))

In [256]:
model = BigramLanguageModel()
print(sum(p.numel() for p in model.parameters()), ' parameters')

optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'dev']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            logits = logits.view(logits.shape[0], -1)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters -1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['dev']:.4f}")

    xb, yb = get_batch('train')

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

209729  parameters
step 0: train loss 4.3975, val loss 4.3932
step 100: train loss 2.6188, val loss 2.6347
step 200: train loss 2.1565, val loss 2.1587
step 300: train loss 1.6311, val loss 1.6401
step 400: train loss 1.0237, val loss 1.0501
step 500: train loss 0.6798, val loss 0.7036
step 600: train loss 0.4934, val loss 0.5266
step 700: train loss 0.3491, val loss 0.3793
step 800: train loss 0.3015, val loss 0.3260
step 900: train loss 0.2630, val loss 0.2889
step 1000: train loss 0.2301, val loss 0.2499
step 1100: train loss 0.2035, val loss 0.2178
step 1200: train loss 0.1572, val loss 0.1759
step 1300: train loss 0.1731, val loss 0.1897
step 1400: train loss 0.1371, val loss 0.1504
step 1500: train loss 0.1327, val loss 0.1523
step 1600: train loss 0.1586, val loss 0.1728
step 1700: train loss 0.1411, val loss 0.1538
step 1800: train loss 0.1151, val loss 0.1246
step 1900: train loss 0.1334, val loss 0.1479
step 2000: train loss 0.1111, val loss 0.1221
step 2100: train loss 0.129

In [258]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long)
print(decode(model.generate(context, max_new_tokens=2000)[0].tolist()))


KVRTTTTTstttttttttsssssTeeee!
Si':
Faves hse.

Als
Alerent
OIo:
Iem.

ILAINII I tast heas sus towh ty toue, s mhter inte terhoe;
That to mehy,
I sat ou at my snoutly I I Therehasat:

Wito sts wool wingew? Towh: athenkor sormy the hamrparryat boz hasthe, yous,
Owomch was ancr amy lu,
I I thifeat yowir uickcond mum indy cuech msu,
Hnous.
EO:
AcI se thin aros yorhaass my soor boult thas, yourht.

I:
I thas youoh ket.

I IIZII ya vee lellhat, thirewels o chen hours thathe orthe hork, sul inmqamy thal.
CEacsus, at nomo,
Hyot
Iklvees:
I, I
Ind tonstain avarese, oug lasw.

It, waye'rojt yulll,
R isb ous meour what hatur, aSedy ye,
'n at blath:-r,r wogse eeheab, monon werelz'UTorord wrow whilts tin haken-outyily isstty ondlern! thel'nss thikes.
Od
TP ther Sin:
Iasks an athas laustay ben to suetsifTele socor This thad ker Hpantes.
O EI
Toumsh ay.

GIEAD: ke ingtan tho.
IV:
Arl:
Watusth,
yaks anged yeve, ate tlll my othed obll ihe yelaob
owtrekl wonsls lamy hot nhe those,
I youlidy,

ARE:
Chtha